In [1]:
import pandas as pd
import re
from collections import defaultdict
import gzip

# Load phenoData

In [2]:
colnames = open("/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_at_scale.phenoData.colnames.txt").read().splitlines()

In [3]:
# loading a test file with the first 5 rows from original phenoData file
df = pd.read_csv("/iblm/netapp/data1/external/Gasperini2019/suppl/test.phenoData.txt",
                header = None, sep = ' ', names = colnames)
df.head()

,sample,cell,total_umis,Size_Factor,gene,all_gene,barcode,read_count,umi_count,proportion,guide_count,sample_directory,ko_barcode_file,id,prep_batch,within_batch_chip,within_chip_lane,percent.mito
0,1A_1_SI-GA-E2,AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,17572,1.009682,chr10.845_top_two_chr1.11183_top_two_chr1.1129...,chr10.845_top_two_chr1.11183_top_two_chr1.1129...,AGAAAGCTCCTCCAGTTCAC_TGATCGCTTTGACTGTGACA_ACAA...,14135,964,0.969819,67,1A_1_SI-GA-E2,guide_libraries/1A_1.gRNAcaptured.txt,1A_1,prep_batch_1,within_batch_chip_A,within_chip_lane_1,0.058787
1,1A_1_SI-GA-E2,AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2,8923,0.939677,chr1.12695_top_two_chr11.3294_top_two_chr1.679...,chr1.12695_top_two_chr11.3294_top_two_chr1.679...,GTAGAGCCTCCAGAACTGTG_AGGTTTATCCAGATGAACTG_CATC...,4329,293,0.844380,26,1A_1_SI-GA-E2,guide_libraries/1A_1.gRNAcaptured.txt,1A_1,prep_batch_1,within_batch_chip_A,within_chip_lane_1,0.036087
2,1A_1_SI-GA-E2,AAACCTGCAAACAACA-1_1A_1_SI-GA-E2,14637,0.990803,ALDH1A2_TSS_BRI3_TSS_chr10.1918_top_two_chr10....,ALDH1A2_TSS_BRI3_TSS_chr10.1918_top_two_chr10....,CCAAGGCGTCCTCAGACCAG_AGCTCCAGGAAGGACCCCCG_TCAC...,12362,884,0.950538,61,1A_1_SI-GA-E2,guide_libraries/1A_1.gRNAcaptured.txt,1A_1,prep_batch_1,within_batch_chip_A,within_chip_lane_1,0.069823
3,1A_1_SI-GA-E2,AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2,22798,1.036578,C16orf91_TSS_chr1.11332_top_two_chr1.1933_top_...,C16orf91_TSS_chr1.11332_top_two_chr1.1933_top_...,GGCGTCAGTCGAGGAGTCAG_GCCAGCACTTCAGCTCACCG_GCTG...,7459,544,0.939551,39,1A_1_SI-GA-E2,guide_libraries/1A_1.gRNAcaptured.txt,1A_1,prep_batch_1,within_batch_chip_A,within_chip_lane_1,0.026187
4,1A_1_SI-GA-E2,AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2,10136,0.952844,chr10.185_top_two_chr10.484_top_two_chr11.4167...,chr10.185_top_two_chr10.484_top_two_chr11.4167...,ATAAGGCACTCACATCCACC_GCTTGTCCCTAACACTCAGA_GGGC...,14831,1054,0.959927,37,1A_1_SI-GA-E2,guide_libraries/1A_1.gRNAcaptured.txt,1A_1,prep_batch_1,within_batch_chip_A,within_chip_lane_1,0.007991


In [4]:
df.shape

(10, 18)

# Use regex to get all gRNA groups observed in each cell

In [8]:
guides = re.findall(r"[A-Za-z0-9]*\_TSS|chr[A-Z0-9]{1,2}\.\w*top_two|chr[A-Z0-9]{1,2}\.\w*second_two", 
                    df['all_gene'].loc[1])
guides

['chr1.12695_top_two',
 'chr11.3294_top_two',
 'chr1.6799_second_two',
 'chr17.695_top_two',
 'chr20.2033_top_two',
 'chr20.314_top_two',
 'chr2.403_second_two',
 'chr2.403_top_two',
 'chr2.4480_top_two',
 'chr2.5077_top_two',
 'chr3.2983_top_two',
 'chr3.5568_top_two',
 'chr5.2905_top_two',
 'chr6.1406_top_two',
 'chr6.1523_second_two',
 'chr6.1669_top_two',
 'chr6.2579_top_two',
 'chr6.4327_top_two',
 'chr6.4750_top_two',
 'chr7.4162_top_two',
 'chr7.4555_top_two',
 'ID2_TSS',
 'MYC_TSS',
 'NXT1_TSS',
 'TMEM14A_TSS',
 'TXN2_TSS']

In [9]:
spacers = df['barcode'].loc[1].split("_")
spacers

['GTAGAGCCTCCAGAACTGTG',
 'AGGTTTATCCAGATGAACTG',
 'CATCTGGTAGCCTCACACCG',
 'GCAGGCTCCCTTAATGCAGG',
 'TTTCTGGCCAATTAGAACCG',
 'GGTCCTGTGCTGTCCCCAAG',
 'GGTCAGCTTGTTTATCCCTG',
 'GCAGCTGCTGGGAATAGCAT',
 'AGCTGGCACCCAGACACTGA',
 'TTAACCACAAAGTAGCATGG',
 'GGAGCAGGTGGGAAACTGGA',
 'TGGCCCCACCAAATGCTGCA',
 'GTGTTTCTGAGCTGTCACCA',
 'AGAAATCTGGGAGAAAGGCG',
 'CATGCCATGGGACTTCCCCA',
 'GGGTGTTGACGACAAACAGA',
 'GTTGCCGTGGTTATATACCG',
 'GCTGATCTGAAACTAAACCG',
 'CTAGCAGTAGTCAGTGGAGG',
 'CAGAACCCATCCCCCACGGA',
 'GGATTCCTCCTGCTAAACGT',
 'CTCACTCACTCCGCCTCTAG',
 'TAATTCCAGCGAGAGGCAGG',
 'CTCGCGTCTGCGTGGAGACG',
 'AGACGGCTGGGCGCCGAGTG',
 'CGACAGGCGTGCCCTTGACG']

### Check if number of spacers and number of guides are equivalent to value in `guide_count` column

In [7]:
len(spacers) == len(guides) == df['guide_count'].loc[1]

True

# Load `GSE120861_grna_groups.at_scale.txt.gz` for sanity check

In [13]:
d = defaultdict(list)

with gzip.open('/iblm/netapp/data1/external/Gasperini2019/suppl/GSE120861_grna_groups.at_scale.txt.gz', 'rt') as fh:
    for l in fh.read().splitlines():
        group, spacer = l.split("\t")
        d[group].append(spacer)

In [16]:
errors = 0
for g,s in zip(guides, spacers):
    if s not in d[g]:
        print("error for gRNA group %s with spacer %s" % (g,s))
        errors += 1
        
if errors==0:
    print("all gRNA groups and corresponding spacers are listed in file")

all gRNA groups and corresponding spacers are listed in file
